In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.utils import shuffle
training_set = './train_simplified/'
from os import listdir
from os.path import isfile, join
train_files = [join(training_set, f) for f in listdir(training_set) if isfile(join(training_set, f))]

In [2]:
def clean_drawing(inkarray):
    inkarray = literal_eval(inkarray)
    stroke_lengths = [len(stroke[0]) for stroke in inkarray]
    total_points = sum(stroke_lengths)
    np_ink = np.zeros((total_points, 3), dtype=np.float32)
    current_t = 0
    for stroke in inkarray:
        for i in [0, 1]:
            np_ink[current_t:(current_t + len(stroke[0])), i] = stroke[i]
        current_t += len(stroke[0])
        np_ink[current_t - 1, 2] = 1  # stroke_end
    # Preprocessing.
    # 1. Size normalization.
    lower = np.min(np_ink[:, 0:2], axis=0)
    upper = np.max(np_ink[:, 0:2], axis=0)
    scale = upper - lower
    scale[scale == 0] = 1
    np_ink[:, 0:2] = (np_ink[:, 0:2] - lower) / scale
    # 2. Compute deltas.
    np_ink = np_ink[1:, 0:2] - np_ink[0:-1, 0:2]
    return np_ink

def load_and_clean(files):
    dfs = []
    for file in files:
        df = pd.read_csv(file)
        dfs += [df]
    full_df = pd.concat(dfs)
    full_df['drawing'] = full_df['drawing'].map(clean_drawing)
    return full_df

In [3]:
df = load_and_clean(train_files[:1])
# randomly shuffle the rows

In [4]:
df

,countrycode,drawing,key_id,recognized,timestamp,word
0,US,"[[-0.22924903, 0.23275864], [-0.1146245, 0.142...",5152802093400064,True,2017-03-08 21:12:07.266040,airplane
1,CA,"[[0.047058824, -0.10169491], [0.07843138, -0.0...",6622904974311424,False,2017-03-06 21:39:04.690130,airplane
2,US,"[[-0.007843137, -0.22368422], [0.027450979, -0...",6577010312740864,True,2017-03-23 02:08:35.229980,airplane
3,US,"[[-0.12941177, 0.0], [-0.13333334, 0.023584902...",5643224746033152,True,2017-03-10 00:35:17.531970,airplane
4,IL,"[[-0.10196079, 0.01321587], [-0.058823533, 0.0...",6670046841536512,True,2017-01-23 18:11:11.658170,airplane
